# GraphLock Cypher

This encryption algorithm introduces a novel approach to securing plaintext by leveraging the principles of **graph coloring**. It constructs a randomized graph based on a derived cryptographic key using PBKDF2 with salt. The nodes of the graph represent ASCII characters, and edges are randomly assigned. The algorithm employs a greedy graph-coloring strategy to assign colors to nodes, where each color serves as a cryptographic parameter. During encryption, each character of the plaintext is encoded into a tuple comprising its assigned color, a computed parameter, and a dynamically generated salt value.  

The encrypted data is serialized and further protected using an XOR-based encryption scheme with the derived key, followed by the integration of an HMAC for integrity verification. Decryption involves reconstructing the graph and its coloring scheme to map the ciphertext back to plaintext. This graph-theoretic approach ensures robust encryption by introducing non-linear transformations and structural randomness, offering enhanced security while maintaining computational efficiency.  

This method is particularly suited for scenarios demanding high randomness and innovative cryptographic paradigms, merging concepts from graph theory and traditional encryption techniques.

1. **`hashlib`**: Provides cryptographic hash functions (e.g., SHA-256) for deriving keys and ensuring data integrity.  
2. **`hmac`**: Adds message authentication to verify data integrity and authenticity using a secure HMAC (Hash-based Message Authentication Code).  
3. **`random`**: Facilitates randomness in graph construction and encryption processes, ensuring non-deterministic outputs.  
4. **`networkx`**: Enables the creation and manipulation of graphs, leveraging properties like node coloring for encryption.  
5. **`os`**: Generates cryptographically secure random bytes (e.g., salts) for deriving secure keys and adding entropy.  


In [1]:
import hashlib
import hmac
import random
import networkx as nx
import os

### `generate_graph` Function

**Purpose:** Creates a randomized graph with ASCII characters (0-127) as nodes, introducing randomness in edge connections.

**Steps:**
1. **`random.seed(seed)`**: Ensures reproducibility for a given seed.
2. **`graph = nx.Graph()`**: Initializes an empty graph.
3. **`ascii_range = range(128)`**: Defines nodes as ASCII characters (0-127).
4. **`graph.add_nodes_from(ascii_range)`**: Adds ASCII nodes.
5. **Edge Creation (`if random.random() < 0.05`)**: Adds edges with a 5% probability between nodes.
6. **Return**: Returns the generated graph.

**Use Case:** Provides a graph structure for encryption algorithms, using its properties (e.g., node coloring) for encoding plaintext.

---

### `randomize_graph` Function

**Purpose:** Randomizes the node order in a graph while ensuring reproducibility with a seed.

**Steps:**
1. **`random.seed(seed)`**: Ensures reproducibility for a given seed.
2. **`nodes = list(graph.nodes())`**: Converts graph nodes to a list.
3. **`random.shuffle(nodes)`**: Shuffles the node list.
4. **`randomized_graph = nx.Graph()`**: Initializes a new empty graph.
5. **`randomized_graph.add_nodes_from(nodes)`**: Adds shuffled nodes.
6. **`randomized_graph.add_edges_from((nodes[i], nodes[j]) for i, j in graph.edges())`**: Adds edges based on shuffled node indices.
7. **Return**: Returns the randomized graph.

**Use Case:** Introduces randomness in the node structure for encryption, maintaining edge connections, ensuring unpredictability with a fixed seed.


In [2]:
def generate_graph(seed: int) -> nx.Graph:
    random.seed(seed)
    graph = nx.Graph()
    ascii_range = range(128)  # Standard ASCII range
    graph.add_nodes_from(ascii_range)
    for i in ascii_range:
        for j in ascii_range:
            if random.random() < 0.05:  # Random edge creation
                graph.add_edge(i, j)
    return graph

def randomize_graph(graph: nx.Graph, seed: int) -> nx.Graph:
    random.seed(seed)
    nodes = list(graph.nodes())
    random.shuffle(nodes)
    randomized_graph = nx.Graph()
    randomized_graph.add_nodes_from(nodes)
    randomized_graph.add_edges_from((nodes[i], nodes[j]) for i, j in graph.edges())
    return randomized_graph

### `derive_key` Function

**Purpose:** Derives a cryptographic key using PBKDF2 with SHA-256, optionally utilizing a provided salt or generating a random one.

**Steps:**
1. **`if salt is None:`**: Checks if a salt is provided. If not, generates a random salt using `os.urandom(16)`.
2. **`salt = os.urandom(16)`**: Generates a random 16-byte salt for key derivation if no salt is provided.
3. **`derived_key = hashlib.pbkdf2_hmac('sha256', key.encode(), salt, 100000)`**: Derives the cryptographic key using PBKDF2 with HMAC-SHA-256. The key is derived from the provided password (`key`), the salt, and 100,000 iterations for enhanced security.
4. **`return int.from_bytes(derived_key, byteorder='big'), salt`**: Converts the derived key (a byte sequence) into an integer and returns it along with the salt.

**Use Case:** Used to securely derive keys from a user-provided password (or passphrase) and an optional salt, ensuring different derived keys even with the same password if the salt differs.


In [3]:
def derive_key(key: str, salt: bytes = None) -> (int, bytes):
    if salt is None:
        salt = os.urandom(16)
    derived_key = hashlib.pbkdf2_hmac('sha256', key.encode(), salt, 100000)
    return int.from_bytes(derived_key, byteorder='big'), salt

### `compute_param` Function

**Purpose:** Computes a cryptographic parameter by combining the ASCII value of a character, the derived key, and a salt, introducing additional randomness.

**Steps:**
1. **`char ^ derived_key ^ salt`**: Performs a bitwise XOR operation between the ASCII value of the character (`char`), the derived cryptographic key (`derived_key`), and a salt value (`salt`). XOR introduces non-linearity, making the parameter more unpredictable.
2. **`% 256`**: The result of the XOR operation is taken modulo 256 to ensure the resulting parameter is within the valid byte range (0 to 255), suitable for cryptographic use.
3. **Return**: Returns the computed parameter, adding extra randomness to the encryption process.

**Use Case:** Used in encryption to compute a unique, randomized parameter for each character in the plaintext, enhancing the security of the overall encryption scheme.


In [4]:
def compute_param(char: int, derived_key: int, salt: int) -> int:
    return (char ^ derived_key ^ salt) % 256

### `xor_encrypt` Function

**Purpose:** Encrypts data using the XOR operation with the derived cryptographic key, providing a simple yet effective encryption method.

**Steps:**
1. **`key_bytes = key.to_bytes((key.bit_length() + 7) // 8, byteorder='big')`**: Converts the derived key (integer) to a byte sequence for XOR operation. The length of the sequence is based on the key's bit-length.
2. **`''.join(chr(ord(c) ^ key_bytes[i % len(key_bytes)]) for i, c in enumerate(data))`**: Iterates over each character in `data`, applies XOR with the corresponding byte of the key, and converts it back to a character. The result is XOR-encrypted data.
3. **Return**: Returns the encrypted data as a string.

**Use Case:** Encrypts serialized data or ciphertext by XOR-ing it with a key, enhancing the encryption scheme.

---

### `add_authentication` Function

**Purpose:** Appends an HMAC (Hash-based Message Authentication Code) to the data to ensure its authenticity and integrity.

**Steps:**
1. **`hmac_digest = hmac.new(auth_key, data, hashlib.sha256).digest()`**: Computes an HMAC using the provided `auth_key` and `data` with SHA-256 for integrity authentication.
2. **`return data + hmac_digest`**: Appends the HMAC digest to the original data, providing both encryption and authentication.

**Use Case:** Authenticates ciphertext to ensure its integrity, preventing tampering during transmission or storage.


In [5]:
def xor_encrypt(data: str, key: int) -> str:
    key_bytes = key.to_bytes((key.bit_length() + 7) // 8, byteorder='big')
    return ''.join(chr(ord(c) ^ key_bytes[i % len(key_bytes)]) for i, c in enumerate(data))

def add_authentication(data: bytes, auth_key: bytes) -> bytes:
    hmac_digest = hmac.new(auth_key, data, hashlib.sha256).digest()
    return data + hmac_digest

### `encrypt` Function

**Purpose:** Encrypts the provided plaintext using a derived cryptographic key and a graph-based mechanism to introduce additional randomness, followed by XOR encryption and HMAC authentication.

**Steps:**
1. **`derived_key, salt = derive_key(key)`**: Derives the cryptographic key using PBKDF2 with the provided password (`key`), generating a derived key and a salt for uniqueness.
2. **`graph = generate_graph(derived_key)`**: Generates a random graph with nodes as ASCII values and edges based on the derived key for structural randomness.
3. **`randomized_graph = randomize_graph(graph, derived_key)`**: Shuffles the graph nodes to add further randomness based on the derived key.
4. **`coloring = nx.coloring.greedy_color(randomized_graph, strategy="random_sequential")`**: Uses a graph coloring algorithm to assign colors to nodes in the randomized graph.
5. **`cipher_pairs = []`**: Iterates over each character in the plaintext:
   - Obtains its ASCII value and corresponding color.
   - Generates a random salt and computes a parameter using the ASCII value, derived key, and salt.
6. **`cipher_str = '|'.join(f"{color}-{param}-{salt}" for color, param, salt in cipher_pairs)`**: Serializes the encrypted character data (color, parameter, salt) into a string.
7. **`encrypted_cipher = xor_encrypt(cipher_str, derived_key)`**: Encrypts the serialized string using XOR with the derived key.
8. **`auth_key = hashlib.sha256(f"{derived_key}".encode()).digest()`**: Derives an HMAC key from the derived key to authenticate the ciphertext.
9. **`final_cipher = add_authentication(encrypted_cipher.encode(), auth_key)`**: Combines the encrypted ciphertext with HMAC for integrity authentication.
10. **Return**: Returns the final ciphertext (with HMAC) as a hexadecimal string, along with the salt used for key derivation.

**Use Case:** Used in the encryption phase of the cryptographic process, where plaintext is securely transformed into ciphertext, with both encryption and integrity authentication using a derived key and graph-based randomness.


In [6]:
def encrypt(plain_text: str, key: str) -> (str, bytes):
    derived_key, salt = derive_key(key)
    graph = generate_graph(derived_key)
    randomized_graph = randomize_graph(graph, derived_key)
    coloring = nx.coloring.greedy_color(randomized_graph, strategy="random_sequential")
    
    cipher_pairs = []
    for char in plain_text:
        ascii_val = ord(char)
        color = coloring[ascii_val]
        salt_value = random.randint(0, 255)  # Random salt for each character
        param = compute_param(ascii_val, derived_key, salt_value)
        cipher_pairs.append((color, param, salt_value))
    
    # Serialize pairs into a string
    cipher_str = '|'.join(f"{color}-{param}-{salt}" for color, param, salt in cipher_pairs)
    
    # Encrypt the serialized string with XOR
    encrypted_cipher = xor_encrypt(cipher_str, derived_key)
    
    # Add HMAC for authentication
    auth_key = hashlib.sha256(f"{derived_key}".encode()).digest()
    final_cipher = add_authentication(encrypted_cipher.encode(), auth_key)
    
    return final_cipher.hex(), salt

### `xor_decrypt` Function

**Purpose:** Decrypts data using the XOR operation with the derived key. XOR encryption is symmetric, so the same function is used for both encryption and decryption.

**Steps:**
1. **`return xor_encrypt(data, key)`**: Since XOR is symmetric, the same `xor_encrypt` function reverses the encryption by applying the XOR operation with the same derived key.

**Use Case:** Used in the decryption phase to reverse XOR encryption and recover the original ciphertext, providing a simple symmetric decryption method.

---

### `decode_character` Function

**Purpose:** Decodes a character from the ciphertext using its associated color, computed parameter, and salt.

**Steps:**
1. **`for char, char_color in coloring.items()`**: Iterates through each character in the coloring map, checking if the color and computed parameter match the encrypted values.
2. **`if char_color == color and compute_param(char, derived_key, salt) == param:`**: If the character's color and parameter match, the function returns the character.
3. **`raise ValueError("Decryption failed. Invalid cipher text or key.")`**: If no match is found, raises an exception indicating decryption failure due to invalid ciphertext or key.

**Use Case:** Used in the decryption process to recover characters from the ciphertext by matching the color, parameter, and salt to the original character.

---

### `verify_authentication` Function

**Purpose:** Verifies the authenticity of the data by comparing the provided HMAC with a freshly computed one to ensure data integrity.

**Steps:**
1. **`received_data, received_hmac = data[:-32], data[-32:]`**: Separates the last 32 bytes as the received HMAC, treating the rest as actual data.
2. **`expected_hmac = hmac.new(auth_key, received_data, hashlib.sha256).digest()`**: Computes a new HMAC from the received data using the `auth_key` and SHA-256.
3. **`if not hmac.compare_digest(received_hmac, expected_hmac):`**: Compares the computed HMAC with the received HMAC to verify authenticity.
4. **`raise ValueError("Authentication failed.")`**: If the HMACs do not match, raises an exception indicating tampering or invalid data.
5. **`return received_data`**: Returns the original data if the HMACs match, confirming data authenticity.

**Use Case:** Used during decryption to verify the integrity of the ciphertext, ensuring it has not been tampered with before proceeding with decryption.


In [7]:
def xor_decrypt(data: str, key: int) -> str:
    return xor_encrypt(data, key)  # XOR is symmetric

def decode_character(color: int, param: int, coloring: dict, derived_key: int, salt: int) -> str:
    for char, char_color in coloring.items():
        if char_color == color and compute_param(char, derived_key, salt) == param:
            return chr(char)
    raise ValueError("Decryption failed. Invalid cipher text or key.")

def verify_authentication(data: bytes, auth_key: bytes) -> bytes:
    """Verify the HMAC and return the data if valid."""
    received_data, received_hmac = data[:-32], data[-32:]
    expected_hmac = hmac.new(auth_key, received_data, hashlib.sha256).digest()
    if not hmac.compare_digest(received_hmac, expected_hmac):
        raise ValueError("Authentication failed.")
    return received_data

### `decrypt` Function

**Purpose:** Decrypts the given ciphertext using a key and salt, recovering the original plaintext by reversing the encryption process.

**Key Steps:**
1. **`derived_key, _ = derive_key(key, salt)`**: Derives a cryptographic key from the provided `key` and `salt` using PBKDF2 to ensure a consistent decryption process.
2. **`auth_key = hashlib.sha256(f"{derived_key}".encode()).digest()`**: Creates an authentication key (`auth_key`) by hashing the `derived_key` with SHA-256, used to verify the integrity of the ciphertext.
3. **`encrypted_data = bytes.fromhex(cipher_text)`**: Converts the hexadecimal `cipher_text` into bytes for further processing.
4. **`decrypted_data = verify_authentication(encrypted_data, auth_key)`**: Verifies the authenticity of the encrypted data using HMAC with the derived `auth_key`. If the HMAC matches, it ensures the data hasn't been tampered with.
5. **`decrypted_str = xor_decrypt(decrypted_data.decode(), derived_key)`**: Uses XOR decryption to revert the encrypted string to its serialized format, undoing the XOR encryption applied during encryption.
6. **`cipher_pairs = [tuple(map(int, pair.split('-'))) for pair in decrypted_str.split('|')]`**: Splits the decrypted string into individual cipher pairs (color, param, and salt value), converting each into a tuple.
7. **`graph = generate_graph(derived_key)`**: Generates a graph based on the `derived_key`, helping with the decoding process based on ASCII characters.
8. **`randomized_graph = randomize_graph(graph, derived_key)`**: Randomizes the graph's node order based on the `derived_key`, ensuring the graph structure is consistent between encryption and decryption.
9. **`coloring = nx.coloring.greedy_color(randomized_graph, strategy="random_sequential")`**: Applies a greedy coloring algorithm to the randomized graph, assigning colors to nodes (characters) for matching encrypted values with their corresponding characters.
10. **`plain_text = []`**: Initializes an empty list to collect decoded characters.
11. **`for color, param, salt_value in cipher_pairs:`**: Iterates through each cipher pair and decodes each character by matching its color, parameter, and salt value with the graph's coloring.
12. **`plain_text.append(decode_character(color, param, coloring, derived_key, salt_value))`**: Decodes each character based on the color, parameter, and salt value and appends it to the `plain_text` list.
13. **`return ''.join(plain_text)`**: Joins the decoded characters into a final string, returning the decrypted plaintext.

**Use Case:** This function is used to decrypt an encrypted message (ciphertext) back to its original form (plaintext). It ensures the integrity and authenticity of the data by checking HMAC values and reversing the cryptographic operations (XOR encryption and graph coloring) applied during encryption.


In [8]:
def decrypt(cipher_text: str, key: str, salt: bytes) -> str:
    """Decrypt the cipher text using the given key."""
    derived_key, _ = derive_key(key, salt)
    auth_key = hashlib.sha256(f"{derived_key}".encode()).digest()
    
    # Verify and extract the ciphertext
    encrypted_data = bytes.fromhex(cipher_text)
    decrypted_data = verify_authentication(encrypted_data, auth_key)
    
    # XOR Decrypt the serialized string
    decrypted_str = xor_decrypt(decrypted_data.decode(), derived_key)
    
    # Reconstruct cipher text pairs
    cipher_pairs = [
        tuple(map(int, pair.split('-'))) for pair in decrypted_str.split('|')
    ]
    
    graph = generate_graph(derived_key)
    randomized_graph = randomize_graph(graph, derived_key)
    coloring = nx.coloring.greedy_color(randomized_graph, strategy="random_sequential")
    
    plain_text = []
    for color, param, salt_value in cipher_pairs:
        plain_text.append(decode_character(color, param, coloring, derived_key, salt_value))
    
    return ''.join(plain_text)

In [9]:
key = "Cryptography"
plain_text = "Hello, Secure World!"

# key = input('Enter Secret Key: ')
# plain_text = input('Enter Plain Text: ')

# Encrypt
cipher_text, salt = encrypt(plain_text, key)
print("Cipher Text:", cipher_text)

# Decrypt
try:
    decrypted_text = decrypt(cipher_text, key, salt)
    print("Decrypted Text:", decrypted_text)
except ValueError as e:
    print(e)

Cipher Text: 53c388c38a0d31c3a8c2a8c3b3c3a5c2bfc2b755c38ac28ac2b760c290c28d3ac3b1c3b4c29fc3ab3c40c28568c2bec3924025354ec39dc38f4532c3a8c2a8c3b3c3bcc3b2c2b64ec287c28fc2ac7ac292c2913fc2bdc3b1c38ec3a9295ec28e70c3b1c29e5e3d375ac388c3810e7ec3b6c2b4c3b9c3a5c3b7c2a949c38fc28bc3bd7dc28fc28e3cc2bfc3a9c283c3a63411c28c70c3b0c3965e3d3250c396c2840d2fc3b6c2afc3a6c3a9c3b6c3b84bc396c28cc2b27bc28fc28e3dc2b5c2b8c284c3b23554c28970c3b0c390596c324ec394c388092fc3b1c2a1c2b7c3a1c3aec2b54dc396c284c2b831c296c2913fc2b4c3b7c29fc3ae335bc3806dc3afc3935b222e51c395c38d4532c3a8c2a8c3bcc3a2c3aec2b648c389c381c2b360c290c28823c2b8c3bcb29a3f187e9eda4b0e4dc14ca02e4816645e805d7b6a71e07de63c0f92fb425e
Decrypted Text: Hello, Secure World!
